https://keras.io/guides/writing_a_training_loop_from_scratch/

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Activation ,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc,precision_score, recall_score,f1_score,accuracy_score
from sklearn.model_selection import StratifiedKFold
from scipy import interp
import matplotlib.pyplot as plt
from skimage import transform
import scipy.ndimage
import matplotlib.pyplot as plt

from keras.layers import Lambda, Input
from keras.models import Model, Sequential
import tensorflow as ktf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
BATCH_SIZE=32
IMG_SIZE=32

In [3]:
import tensorflow_datasets as tfds
train_ds = tfds.load(name="cifar10",split=tfds.Split.TRAIN)
test_ds = tfds.load(name="cifar10",split=tfds.Split.TEST)

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteE0Y3K8/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteE0Y3K8/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [8]:
def scale_image(features):
    image = tf.cast(features['image'], tf.float32)
    image /= 255

    return image, features['label']

In [9]:
train_batches = train_ds.map(scale_image).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

test_batches = test_ds.map(scale_image).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
# plotting helper function
def plothist(hist):
    
    plt.subplot(1, 2, 1)
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')


    plt.show()

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-twnq2aux
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-twnq2aux
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20030 sha256=c67e6bd3d181f41ce1f8bc1271b0d80bb53511a28ff067d6dd36d2a2bd4ce275
  Stored in directory: /tmp/pip-ephem-wheel-cache-d4963i8b/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [ ]:
from classification_models.tfkeras import Classifiers

In [ ]:
ResNet18, preprocess_input = Classifiers.get('resnet18')
resnet18 = ResNet18((224, 224, 3), weights='imagenet')
#resnet18.summary() 

In [ ]:
alpha = 0.003  # weight decay coefficient

for layer in resnet18.layers:
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.Dense):
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.kernel))
    if hasattr(layer, 'bias_regularizer') and layer.use_bias:
        layer.add_loss(lambda: keras.regularizers.l2(alpha)(layer.bias))

In [ ]:
x = resnet18.layers[-2].output
output = Dense(units=10,activation='softmax')(x)
model = Model(inputs=[resnet18.input], outputs=[output])
#model.summary()

In [ ]:
for layer in model.layers[35:]:
    layer.trainable = True
for layer in model.layers[:35]:
    layer.trainable = False
#for i, layer in enumerate(model.layers):
    #print(i, layer.name, layer.trainable)

In [ ]:
newInput = Input(batch_shape=(None, 32, 32, 3))
resizedImg = Lambda(lambda image: ktf.image.resize(image, (224, 224)))(newInput)
newOutputs = model(resizedImg)
model = Model(newInput, newOutputs)

In [ ]:
BATCH_SIZE =100
STEPS_PER_EPOCH =len(train_batches)//BATCH_SIZE 

#step = tf.Variable(0, trainable=False)
boundaries = [30*STEPS_PER_EPOCH,50*STEPS_PER_EPOCH,80*STEPS_PER_EPOCH]
values = [ 0.1, 0.01,0.001,0.0001]
learning_rate_fn = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries, values)

In [ ]:
opt = SGD(learning_rate=learning_rate_fn)#,momentum=0.9,decay=0.01,nesterov=False
#opt = Adam(learning_rate=0.0001,beta_1=0.9,beta_2=0.99,epsilon=0.1,amsgrad=False,name="Adam",)
#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

In [ ]:
# Instantiate an optimizer.
optimizer = opt
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
import time

epochs = 50
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_batches):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn( y_batch_train, logits)


        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * 64))


    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_test, y_batch_test in test_batches:
        val_logits = model(x_batch_test, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_test, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    


Start of epoch 0
Training loss (for one batch) at step 0: 2.8809
Seen so far: 64 samples
Training loss (for one batch) at step 200: 2.2909
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 2.3080
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 2.2627
Seen so far: 38464 samples
Training loss (for one batch) at step 800: 2.2477
Seen so far: 51264 samples
Training loss (for one batch) at step 1000: 2.3095
Seen so far: 64064 samples
Training loss (for one batch) at step 1200: 2.3209
Seen so far: 76864 samples
Training loss (for one batch) at step 1400: 2.2306
Seen so far: 89664 samples
Training acc over epoch: 0.1357
Validation acc: 0.1477

Start of epoch 1
Training loss (for one batch) at step 0: 2.2505
Seen so far: 64 samples
Training loss (for one batch) at step 200: 2.2886
Seen so far: 12864 samples
Training loss (for one batch) at step 400: 2.2769
Seen so far: 25664 samples
Training loss (for one batch) at step 600: 2.2029
Seen so far: 38464 